# Gerador de Perguntas Dinâmico Utilizando T5 e PTT5 em Português/Inglês 
## Parte 3 - Gerando Perguntas

*Baseado no incrível trabalho realizado por Suraj Patil* -  [Question Generation 
Using Transformers](https://github.com/patil-suraj/question_generation)  

--- 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install nlp
!pip install sentencepiece
!pip install datasets

In [3]:
import os

import torch

from transformers import(
    AutoModelForSeq2SeqLM, 
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    AutoConfig,
    T5Tokenizer
)

In [4]:
GDRIVE_PATH = '/content/drive/MyDrive'

MODEL_PTT5_BASE = 'unicamp-dl/ptt5-base-portuguese-vocab'
MODEL_PTT5_LARGE = 'unicamp-dl/ptt5-large-portuguese-vocab'
MODEL_T5_BASE= "t5-base"

MODEL_ROOT_PATH = os.path.join(GDRIVE_PATH, 'model')

MAX_SOURCE_LENGTH=512
MAX_TARGET_LENGTH=32

In [5]:
class E2EQGPipeline:
    def __init__(
        self,
        model_path_or_name: str,
        tokenizer_path_or_name: str,
        use_cuda: bool
    ) :

        self.model:PreTrainedModel = AutoModelForSeq2SeqLM.from_pretrained(model_path_or_name)
        
        self.tokenizer:PreTrainedTokenizer = T5Tokenizer.from_pretrained(tokenizer_path_or_name, config=AutoConfig.from_pretrained(model_path_or_name))

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        assert self.model.__class__.__name__ in ["T5ForConditionalGeneration", "BartForConditionalGeneration"]
        
        self.model_type = "t5"
        
        self.default_generate_kwargs = {
            "max_length": 256,
            "num_beams": 4,
            "length_penalty": 1.5,
            "no_repeat_ngram_size": 3,
            "early_stopping": True,
        }
    
    def __call__(self, context: str, **generate_kwargs):
        inputs = self._prepare_inputs_for_e2e_qg(context)

        # TODO: when overrding default_generate_kwargs all other arguments need to be passsed
        # find a better way to do this
        if not generate_kwargs:
            generate_kwargs = self.default_generate_kwargs
        
        input_length = inputs["input_ids"].shape[-1]
        
        # max_length = generate_kwargs.get("max_length", 256)
        # if input_length < max_length:
        #     logger.warning(
        #         "Your max_length is set to {}, but you input_length is only {}. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)".format(
        #             max_length, input_length
        #         )
        #     )

        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device),
            **generate_kwargs
        )

        prediction = self.tokenizer.decode(outs[0], skip_special_tokens=True)
        questions = prediction.split("<sep>")
        questions = [question.strip() for question in questions[:-1]]
        return questions
    
    def _prepare_inputs_for_e2e_qg(self, context):
        source_text = f"gerar perguntas: {context}"
        source_text = source_text + " </s>"
        
        inputs = self._tokenize([source_text], padding=False)
        return inputs
    
    def _tokenize(
        self,
        inputs,
        padding=True,
        truncation=True,
        add_special_tokens=True,
        max_length=512
    ):
        inputs = self.tokenizer.batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs


### Modelo em Português

In [9]:
model_path_pt_br = MODEL_PTT5_BASE #MODEL_T5_BASE #MODEL_PTT5_LARGE
model_name_pt_br = model_path_pt_br.split(sep="/")[-1]
model_path_pt_br = os.path.join(MODEL_ROOT_PATH, model_name_pt_br + '-qg')

e2e_qg_pt_br = E2EQGPipeline(model_path_or_name=model_path_pt_br,
                       tokenizer_path_or_name=model_path_pt_br,
                       use_cuda=True)

##### Textos em portugês para geração de perguntas

In [10]:
texts_pt_br = []

texts_pt_br.append("Python é uma linguagem de programação interpretada, de alto nível e de uso geral. Criado por Guido van Rossum \
e lançado pela primeira vez em 1991, a filosofia de design do Python enfatiza o código \
legibilidade com seu uso notável de espaços em branco significativos.")

texts_pt_br.append("Gravidade do latim gravitas, que significa 'peso', ou gravitação, é um fenômeno natural pelo qual todos \
coisas com massa ou energia, incluindo planetas, estrelas, galáxias e até mesmo luz são trazidas para  \
um outro. Na Terra, a gravidade dá peso aos objetos físicos, e a gravidade da Lua causa as marés do oceano. \
A atração gravitacional da matéria gasosa original presente no Universo fez com que ela começasse a coalescer \
e formando estrelas e fazendo com que as estrelas se agrupassem em galáxias, então a gravidade é responsável por muitos de \
as estruturas em grande escala do Universo. A gravidade tem alcance infinito, embora seus efeitos sejam cada vez mais \
mais fraco à medida que os objetos se distanciam")

texts_pt_br.append("42 é a resposta para a vida, universo e tudo.")

texts_pt_br.append("Forrest Gump é um filme de comédia e drama americano de 1994 dirigido por Robert Zemeckis e escrito por Eric Roth. \
É baseado no romance homônimo de 1986 de Winston Groom e é estrelado por Tom Hanks, Robin Wright, Gary Sinise, \
Mykelti Williamson e Sally Field. A história retrata várias décadas da vida de Forrest Gump , \
um homem de raciocínio lento, mas de bom coração, do Alabama, que testemunha e influencia inconscientemente várias \
eventos históricos nos Estados Unidos do século XX. O filme difere substancialmente do romance.")

texts_pt_br.append("Arquitetonicamente, a escola tem um car\u00e1ter cat\u00f3lico. No topo da c\u00fapula de ouro do edif\u00edcio principal est\u00e1 uma est\u00e1tua de ouro da Virgem Maria. \
                    Imediatamente em frente ao edif\u00edcio principal e de frente para ele, est\u00e1 uma est\u00e1tua de Cristo de cobre com os bra\u00e7os erguidos com a lenda \"Venite Ad Me Omnes\". \
                    Ao lado do edif\u00edcio principal \u00e9 a Bas\u00edlica do Sagrado Cora\u00e7\u00e3o. Imediatamente atr\u00e1s da bas\u00edlica fica a Gruta, um local mariano de ora\u00e7\u00e3o e reflex\u00e3o. \
                    \u00c9 uma r\u00e9plica da gruta de Lourdes, na Fran\u00e7a, onde a Virgem Maria apareceu para Santa Bernadette Soubirous em 1858. \
                    No final da estrada principal (e em uma linha direta que liga tr\u00eas est\u00e1tuas ao Domo de Ouro), existe uma est\u00e1tua de pedra simples e moderna de Maria")

texts_pt_br.append("Tétano é uma infeção bacteriana grave caracterizada por espasmos musculares. \
         No tipo mais comum, os espasmos têm início no maxilar e progridem para o resto do corpo. \
         Os episódios de espasmos têm geralmente a duração de alguns minutos e ocorrem com frequência durante três ou quatro semanas. \
         Os espasmos podem ser de tal forma intensos que podem provocar fraturas ósseas. \
         Os outros sintomas podem incluir febre, sudação, dor de cabeça, dificuldade ao engolir, hipertensão e aumento do ritmo cardíaco. \
         Os sintomas geralmente manifestam-se entre três a vinte e dois dias após a infeção. O recobro pode levar meses. Cerca de 10% das pessoas infectadas morrem.")

texts_pt_br.append("Os ventos alísios são ventos que ocorrem durante todo o mês nas regiões sub-tropicais, sendo muito comuns na América Central. \
         São o resultado da ascensão de massas de ar que convergem de zonas de alta pressão (anticiclónicas), nos trópicos, para zonas de baixa pressão (ciclónicas) no Equador, formando um ciclo. \
         São ventos úmidos, que provocam chuvas nos locais onde convergem. Por essa razão, a zona equatorial é a região das calmarias equatoriais chuvosas. \
         O Alísio de hemisfério Norte sopra de Nordeste para Sudoeste, enquanto o do hemisfério Sul sopra do Sudeste para o Noroeste. \
         A sua influência é mais marcante no clima de regiões costeiras e de baixa latitude, exercendo grande importância na meteorologia insular." )


In [11]:
for texto in texts_pt_br:
    print('Texto: ', texto)
    print('Perguntas :', e2e_qg_pt_br(texto))
    print('------------------------------------')

Texto:  Python é uma linguagem de programação interpretada, de alto nível e de uso geral. Criado por Guido van Rossum e lançado pela primeira vez em 1991, a filosofia de design do Python enfatiza o código legibilidade com seu uso notável de espaços em branco significativos.


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Perguntas : ['Quem criou o Python?', 'Em que ano o Python foi lançado pela primeira vez?']
------------------------------------
Texto:  Gravidade do latim gravitas, que significa 'peso', ou gravitação, é um fenômeno natural pelo qual todos coisas com massa ou energia, incluindo planetas, estrelas, galáxias e até mesmo luz são trazidas para  um outro. Na Terra, a gravidade dá peso aos objetos físicos, e a gravidade da Lua causa as marés do oceano. A atração gravitacional da matéria gasosa original presente no Universo fez com que ela começasse a coalescer e formando estrelas e fazendo com que as estrelas se agrupassem em galáxias, então a gravidade é responsável por muitos de as estruturas em grande escala do Universo. A gravidade tem alcance infinito, embora seus efeitos sejam cada vez mais mais fraco à medida que os objetos se distanciam
Perguntas : ['Qual é o termo para gravitação?']
------------------------------------
Texto:  42 é a resposta para a vida, universo e tudo.
Perguntas 

### Modelo em Inglês

In [12]:
model_path_en = MODEL_T5_BASE
model_name_en = model_path_en.split(sep="/")[-1]
model_path_en = os.path.join(MODEL_ROOT_PATH, model_name_en + '-qg')

e2e_qg_en = E2EQGPipeline(model_path_or_name=model_path_en,
                       tokenizer_path_or_name=model_path_en,
                       use_cuda=True)

##### Textos em Inglês

In [13]:
texts_en = []

texts_en.append("Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum \
and first released in 1991, Python's design philosophy emphasizes code \
readability with its notable use of significant whitespace.")

texts_en.append("Gravity (from Latin gravitas, meaning 'weight'), or gravitation, is a natural phenomenon by which all \
things with mass or energy—including planets, stars, galaxies, and even light—are brought toward (or gravitate toward) \
one another. On Earth, gravity gives weight to physical objects, and the Moon's gravity causes the ocean tides. \
The gravitational attraction of the original gaseous matter present in the Universe caused it to begin coalescing \
and forming stars and caused the stars to group together into galaxies, so gravity is responsible for many of \
the large-scale structures in the Universe. Gravity has an infinite range, although its effects become increasingly \
weaker as objects get further away")

texts_en.append("42 is the answer to life, universe and everything.")

texts_en.append("Forrest Gump is a 1994 American comedy-drama film directed by Robert Zemeckis and written by Eric Roth. \
It is based on the 1986 novel of the same name by Winston Groom and stars Tom Hanks, Robin Wright, Gary Sinise, \
Mykelti Williamson and Sally Field. The story depicts several decades in the life of Forrest Gump (Hanks), \
a slow-witted but kind-hearted man from Alabama who witnesses and unwittingly influences several defining \
historical events in the 20th century United States. The film differs substantially from the novel.")

In [14]:
for texto in texts_en:
    print('Texto: ', texto)
    print('Perguntas :', e2e_qg_en(texto))
    print('------------------------------------')

Texto:  Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace.


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Perguntas : ['Who created Python?', 'When was Python first released?', "What does Python's design philosophy emphasize?", 'How is Python code readable?']
------------------------------------
Texto:  Gravity (from Latin gravitas, meaning 'weight'), or gravitation, is a natural phenomenon by which all things with mass or energy—including planets, stars, galaxies, and even light—are brought toward (or gravitate toward) one another. On Earth, gravity gives weight to physical objects, and the Moon's gravity causes the ocean tides. The gravitational attraction of the original gaseous matter present in the Universe caused it to begin coalescing and forming stars and caused the stars to group together into galaxies, so gravity is responsible for many of the large-scale structures in the Universe. Gravity has an infinite range, although its effects become increasingly weaker as objects get further away
Perguntas : ['What is the Latin word for gravitas?', 'What does gravity do on Earth?']
------